In [12]:
from tqdm import tqdm
import pandas as pd
from rank_bm25 import BM25Okapi
from eunjeon import Mecab  # KoNLPy style mecab wrapper

In [2]:
excel_data = pd.read_excel('naver_it2.xlsx', engine ='openpyxl')

In [3]:
excel_data

,Unnamed: 0,source,cid,categoryId,docId,title,main
0,0,https://terms.naver.com,59088,59088,3573766,MP4,"인터넷 시대에 발맞춰 태어난 멀티미디어 파일 규격, MP4윈도우(Windows)나도..."
1,1,https://terms.naver.com,59088,59088,5148680,HEIF(고효율 이미지 파일 형식),HEIF는 '고효율 이미지 파일 형식(High Efficiency Image Fil...
2,2,https://terms.naver.com,59088,59088,3579922,iOS9,‘iOS9’애플이 2015년 9월 17일부터 모바일 기기용 운영체제(OS) ‘iOS...
3,3,https://terms.naver.com,59088,59088,3577875,무선 디스플레이 전송 기술,"모바일 기기의 성능은 비약적으로 발전하고 있지만, 여전히 화면이 작다는 것은 치명적..."
4,4,https://terms.naver.com,59088,59088,3575828,캐리어 어그리게이션,LTE에서는 여러 개의 주파수를 섞어서 더 빠른 속도를 내는 기술이 적용된다.이동통...
...,...,...,...,...,...,...,...
33738,33738,https://terms.naver.com,40942,32827,1179640,콜백,상대방에게 국제전화를 되걸어주는 특수 서비스.국제전화요금이국가에 따라 다르게 적용되...
33739,33739,https://terms.naver.com,40942,32827,1179641,CSS,웹 문서의 전반적인 스타일을 미리 저장해 둔 스타일시트이다. 문서 전체의 일관성을 ...
33740,33740,https://terms.naver.com,40942,32827,1179642,CMS,"은행에 직접 가지 않아도 통신과 컴퓨터를 이용하여 은행 구좌를 살필 수 있고, 주식..."
33741,33741,https://terms.naver.com,40942,32827,1064955,광석라디오,"방연석·실리콘·저마늄 등의 광석을 이용한, 동조회로·검파기 및 리시버로 이루어진 매..."


In [4]:
corpus = excel_data['title'] +", "+ excel_data['main'] 
print(corpus)

0        MP4, 인터넷 시대에 발맞춰 태어난 멀티미디어 파일 규격, MP4윈도우(Windo...
1        HEIF(고효율 이미지 파일 형식), HEIF는 '고효율 이미지 파일 형식(High...
2        iOS9, ‘iOS9’애플이 2015년 9월 17일부터 모바일 기기용 운영체제(OS...
3        무선 디스플레이 전송 기술, 모바일 기기의 성능은 비약적으로 발전하고 있지만, 여전...
4        캐리어 어그리게이션, LTE에서는 여러 개의 주파수를 섞어서 더 빠른 속도를 내는 ...
                               ...                        
33738    콜백, 상대방에게 국제전화를 되걸어주는 특수 서비스.국제전화요금이국가에 따라 다르게...
33739    CSS, 웹 문서의 전반적인 스타일을 미리 저장해 둔 스타일시트이다. 문서 전체의 ...
33740    CMS, 은행에 직접 가지 않아도 통신과 컴퓨터를 이용하여 은행 구좌를 살필 수 있...
33741    광석라디오, 방연석·실리콘·저마늄 등의 광석을 이용한, 동조회로·검파기 및 리시버로...
33742    CISA, 정보시스템 감사 컨트롤 협회(ISACA)가 인증하는, 시스템을 독자적으로...
Length: 33743, dtype: object


In [5]:
tagger = Mecab(dicpath='C:/mecab/mecab-ko-dic')
def tokenizer(sent):
  return tagger.morphs(sent)

In [13]:
tokenized_corpus = [tokenizer(doc) for doc in tqdm(corpus)]
print(tokenized_corpus[:10])

100%|██████████████████████████████████████████████████████████████████████████| 33743/33743 [00:12<00:00, 2617.69it/s]

[['MP', '4', ',', '인터넷', '시대', '에', '발맞춰', '태어난', '멀티미디어', '파일', '규격', ',', 'MP', '4', '윈도우', '(', 'Windows', ')', '나', '도스', '(', 'Dos', ')', '와', '같', '은', '컴퓨터', '운영', '체제', '에서', '는', '파', '일', '명', '뒤', '에', '붙', '는', '확장자', '(', 'Filename', 'extension', ')', '로', '해당', '파일', '의', '쓰임새', '를', '구분', '하', '곤', '한다', '.', '예', '를', '들', '어', '같', '은', '‘', '대한민국', '’', '이', '라는', '이름', '을', '가진', '파일', '이', '라도', '그', '뒤', '에', '.', 'jpg', '가', '붙', '어', '있', '다면', '그림', '파일', ',', '.', 'txt', '가', '붙', '어', '있', '다면', '문서', '파일', '이', '다', '.', '이런', '파일', '규격', '을', '정하', '는', '것', '은', '해당', '파일', '의', '규격', '을', '처음', '개발', '한', '개발', '사나', '개발자', '다', '.', '특히', '동영상', '이나', '음성', '과', '같', '은', '멀티미디어', '관련', '파일', '규격', '을', '많이', '개발', '한', '대표', '적', '인', '곳', '이', '바로', 'MPEG', '(', 'Moving', 'Picture', 'Experts', 'Group', ',', '엠펙', ')', '이', '라는', '단체', '다', '.', 'MPEG', '는', '동영상', '기술', '연구', '를', '하', '는', '전문가', '들', '이', '모여', '1988', '년', '에', '설립', '했으며', ',', '여기'

In [14]:
bm25 = BM25Okapi(tokenized_corpus)

In [15]:
def getNouns(query):
    return tagger.nouns(query)

In [26]:
def getDocs(query):
    return bm25.get_top_n(getNouns(query), corpus, n=5)

In [25]:
getDocs("자연어 처리가뭐야?")[0]

'자연어 처리, 컴퓨터를 이용해 사람의 자연어를 분석하고 처리하는 기술.약어NLP요소 기술로 자연어 분석, 이해, 생성 등이 있으며,정보 검색,기계 번역, 질의응답 등 다양한 분야에 응용된다.자연어는 일반 사회에서 자연히 발생하여 사람이 의사소통에 사용하는 언어로, 컴퓨터에서 사용하는 프로그래밍 언어와 같이 사람이 의도적으로 만든 인공어(constructed language)에 대비되는 개념이다.자연어 처리에는 자연어 분석, 자연어 이해, 자연어 생성 등의 기술이 사용된다. 자연어 분석은 그 정도에 따라 형태소 분석(morphological analysis), 통사 분석(syntactic analysis),의미 분석(semantic analysis) 및 화용 분석(pragmatic analysis)의 4 가지로 나눌 수 있다. 자연어 이해는 컴퓨터가 자연어로 주어진 입력에 따라 동작하게 하는 기술이며, 자연어 생성은 동영상이나 표의 내용 등을 사람이 이해할 수 있는 자연어로 변환하는 기술이다.자연어 처리는인공 지능의 주요 분야 중 하나로, 1950년대부터 기계 번역과 같은 자연어 처리 기술이 연구되기 시작했다. 1990년대 이후에는 대량의 말뭉치(corpus) 데이터를 활용하는기계 학습기반 및 통계적 자연어 처리 기법이 주류가 되었으며, 최근에는심층 기계 학습(deep learning) 기술이 기계 번역 및 자연어 생성 등에 적용되고 있다.※ 통사 분석(syntactic anaylisys): 컴퓨터 분야에서는 ‘구문 해석’이라 하나 언어학에서는 ‘통사 분석’이라 함. 통사는 생각이나 감정을 말과 글로 표현할 때 완결된 내용을 나타내는 최소의 단위.※ 통사: 주어와 서술어를 갖추고 있는 것이 원칙이나 때로 이런 것이 생략될 수도 있다. 글의 경우, 문장의 끝에 ‘.’, ‘?’, ‘!’ 따위의 마침표를 찍는다. ‘철수는 몇 살이니?’, ‘세 살.’, ‘정말?’ 따위이다. (출처: 우리말샘)※ 화용(話用) 분석: 말하는 이, 듣는 이, 시간, 장소 따위로 구성되는 